In [1]:
!pip install timm --upgrade
!pip install accelerate
!pip install git+https://github.com/huggingface/transformers.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 68.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 89.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.9.41
    Uninstalling nvidia-nvjitlink-cu12-12.9.41:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.9.41
  Attempting uninstall: nvidia-curand-cu12
    Found existing insta

# Download Gemma-3n

In [2]:
import kagglehub

GEMMA_PATH = kagglehub.model_download("google/gemma-3n/transformers/gemma-3n-e2b-it")

# Use Gemma-3n for text-generation

In [3]:
import transformers
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

tokenizer = AutoTokenizer.from_pretrained(GEMMA_PATH, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(GEMMA_PATH, trust_remote_code=True)
prompt = "Why are there so many Geese on Kaggle?"
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
generation_config = GenerationConfig(max_new_tokens=150, do_sample=True, temperature=0.7)
outputs = model.generate(**inputs, generation_config=generation_config)
result = tokenizer.decode(outputs[0], skip_special_tokens=True)

2025-06-28 09:50:07.407686: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751104207.604833      20 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751104207.663622      20 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

`generation_config` default values have been modified to match model-specific defaults: {'cache_implementation': 'hybrid', 'top_k': 64, 'top_p': 0.95, 'pad_token_id': 0, 'bos_token_id': 2, 'eos_token_id': [1, 106]}. If this is not desired, please set these values explicitly.


In [4]:
print(result)

Why are there so many Geese on Kaggle?

This is a funny question! It's a playful observation, and it's not something that has a straightforward, definitive answer. However, here's a breakdown of why you might be seeing a lot of "Geese" in the context of Kaggle, and why it's a popular (and sometimes humorous) username choice:

**1. The "Geese" Community/Culture:**

*   **Community and Collaboration:**  There's a strong sense of community among many Kaggle users, and "Geese" has become a somewhat recognized shorthand for this group.  It's a friendly, approachable, and collaborative username.
*   **Positive Association:**  Geese are generally seen as symbols of


# Use Gemma-3n for image-understanding

In [5]:
from IPython.display import Image
IMAGE_URL="https://storage.googleapis.com/kaggle-media/competitions/question_goose.png"
Image(url=IMAGE_URL,height=250,width=250)

In [6]:
from transformers import AutoProcessor, AutoModelForImageTextToText

processor = AutoProcessor.from_pretrained(GEMMA_PATH)
model = AutoModelForImageTextToText.from_pretrained(GEMMA_PATH, torch_dtype="auto", device_map="auto")

messages = [
    {
        "role": "user",
        "content": [
            {"type": "image", "image": IMAGE_URL},
            {"type": "text", "text": "Describe this image in detail."}
        ]
    }
]

inputs = processor.apply_chat_template(
    messages,
    add_generation_prompt=True,
    tokenize=True,
    return_dict=True,
    return_tensors="pt"
).to(model.device, dtype=model.dtype)
input_len = inputs["input_ids"].shape[-1]

outputs = model.generate(**inputs, max_new_tokens=512, disable_compile=True)
text = processor.batch_decode(
    outputs[:, input_len:],
    skip_special_tokens=True,
    clean_up_tokenization_spaces=True
)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [7]:
print(text[0])

The image is a black and white cartoon illustration depicting a stylized, elongated creature that resembles a whale or a large, sleek animal. It is set against a solid light blue, slightly rounded background.

The creature's body is predominantly white with a black outline. It has a long, curved neck that is bent forward, and a large head. The head has small, simple black eyes and a curved, smiling mouth.  A single, small black horn protrudes from the top of its head.

The creature's body seems to be intertwined with a series of curved, segmented shapes that resemble tubes or coils, possibly representing internal organs or movement. These shapes are also white with black outlines and appear to be wrapping around and through the creature's body.

At the end of the creature's body, near the rear, there are two large, rectangular blocks that have some dimension to them, suggesting a solid form. These blocks are also white with black outlines.

An arrow points from the creature's head towa

In [8]:
# Silly Explainer Loop
# Need Cuda
# enter topics repeatedly
'''
while True:
    topic = input("Enter a topic ('quit' to exit): ")
    if topic.lower() == "quit":
        print("Goodbye from the Luffy! 👒")
        break

    #  funny and incorrect prompt
    prompt = f"Explain {topic} like I'm five... but get some things hilariously wrong, as if you're a dumb who read half a science book. Make sure you don't tell exact answer"

    # Tokenize input
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    #  generation settings for creative/wrong answers
    generation_config = GenerationConfig(
        max_new_tokens=200,
        do_sample=True,
        temperature=1.0,  # more randomness
        top_k=50,
        top_p=0.95
    )

    # Generate the response
    outputs = model.generate(**inputs, generation_config=generation_config)

    # Decode the generated output
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)

    print("\n🧪 Gemma Explains:", result, "\n")
'''

'\nwhile True:\n    topic = input("Enter a topic (\'quit\' to exit): ")\n    if topic.lower() == "quit":\n        print("Goodbye from the Luffy! 👒")\n        break\n\n    #  funny and incorrect prompt\n    prompt = f"Explain {topic} like I\'m five... but get some things hilariously wrong, as if you\'re a dumb who read half a science book. Make sure you don\'t tell exact answer"\n\n    # Tokenize input\n    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)\n\n    #  generation settings for creative/wrong answers\n    generation_config = GenerationConfig(\n        max_new_tokens=200,\n        do_sample=True,\n        temperature=1.0,  # more randomness\n        top_k=50,\n        top_p=0.95\n    )\n\n    # Generate the response\n    outputs = model.generate(**inputs, generation_config=generation_config)\n\n    # Decode the generated output\n    result = tokenizer.decode(outputs[0], skip_special_tokens=True)\n\n    print("\n🧪 Gemma Explains:", result, "\n")\n'